In [1]:
import csv
import os
import requests
import re
import orodja


# Najprej definirajmo nekaj pomožnih orodij za pridobivanje podatkov s spleta.

## Definirajte URL glavne strani bolhe za oglase z mačkami

In [2]:
cats_frontpage_url = 'http://www.bolha.com/zivali/male-zivali/macke/'

Mapa, v katero bomo shranili podatke

In [3]:
cat_directory = 'macke'

Ime datoteke v katero bomo shranili glavno stran

In [4]:
frontpage_filename = 'macke_frontpage.html'

Ime CSV datoteke v katero bomo shranili podatke

In [5]:
csv_filename = 'macke.csv'

Funkcija, ki sprejme URL spletne strani kot argument in spletno stran poskuša prenesti z pomočjo *requests*. Stran z vsebino vrne v obliki niza.

In [6]:
def download_url_to_string(url):
    try:
        page_content = requests.get(url).text()
        # del kode, ki morda sproži napako
  
    except requests.exceptions.RequestException as e :
        # koda, ki se izvede pri napaki
        # dovolj je če izpišemo opozorilo in prekinemo izvajanje funkcije
        print(e)
        page_content =""
    # nadaljujemo s kodo če ni prišlo do napake
    return page_content

Funkcija zapiše vrednost parametra *text* v novo ustvarjeno datoteko
    locirano v *directory/filename*, ali povozi obstoječo. V primeru, da je
    niz *directory* prazen datoteko ustvari v trenutni mapi.

In [7]:
def save_string_to_file(text, directory, filename):
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8') as file_out:
        file_out.write(text)
    return None

## Definirajte funkcijo, ki prenese glavno stran in jo shrani v datoteko.

In [8]:
def save_frontpage(url, directory, filename):
    '''Save "cats_frontpage_url" to the file
    "cat_directory"/"frontpage_filename"'''
    
    content = download_url_to_string(url)
    directory = cat_directory
    filename = csv_filename
    save_string_to_file(content, directory, filename)
    return None

## Po pridobitvi podatkov jih želimo obdelati.

In [9]:
def read_file_to_string(directory, filename):
    '''Return the contents of the file "directory"/"filename" as a string.'''
    
    with open (csv_filename, 'r') as catsfile:
        data = catsfile.read()
    return data

## Definirajte funkcijo, ki sprejme niz, ki predstavlja vsebino spletne strani, in ga razdeli na dele, kjer vsak del predstavlja en oglas. 
## To storite s pomočjo regularnih izrazov, ki označujejo začetek in konec posameznega oglasa. 
## Funkcija naj vrne seznam nizov.

*re.compile(regex_izraz,text)* tvori niz podatkov, definira od kje do kje je en oglas, 
t.j.: od *div clas=ad* do *div clas=clear*.
Dodamo oznake *flags=re.DOTALL* da z regex znakom *.* ki predstavlja vsak splošen znak , tudi presledek, zajamemo še značko za novo vrtico *\n*.
(...da nam ne povzočajo težav nove vrstice)

*re.findall()* vrne seznam nizov.

In [10]:
def page_to_ads(page_content):
    '''Split "page" to a list of advertisement blocks.'''
  
    oglas = re.compile(
        r'<dic clas ="ad">(.*?)<div clas = "clear">', flags = re.DOTALL
        )
    oglasi = re.findall(oglas, page_content)
    return oglasi


## Definirajte funkcijo, ki sprejme niz, ki predstavlja oglas, in izlušči podatke o imenu, ceni in opisu v oglasu.

*re.search(pattern, string, flags=0)* ... pojdi čez niz in poišči vzorec, ki se ujema. Vrni *None* , če se nič ne ujema.
*.groupdict()* .... ko najdeš vzorec, ki se ujema, sestavi slovar iz podatkov.

In [11]:
def get_dict_from_ad_block(oglas):
    '''Build a dictionary containing the name, description and price
    of an ad block.'''
    
    vzorec_oglasa = re.compile(
        r'<h3><a title = "(?P<ime>.+?)"'
        r'href = "(?P<link>.+?)"'
        r'class = "additionalInfo"><span><b>(?P<rodovnik>.*?)</b></span><div/>'
        r'<div class = "price"><span>(?P<cena>.*?)</span></div>',
        flags = re.DOTALL
    )
    data = re.search(vzorec_oglasa, oglas)
    slovar_oglasov = data.groupdict()


    return slovar_oglasov

## Definirajte funkcijo, ki sprejme ime in lokacijo datoteke, ki vsebuje besedilo spletne strani, in vrne seznam slovarjev, ki vsebujejo podatke o vseh oglasih strani.

In [12]:
def ads_from_file(filename, directory):
    
    stran_s_podatki = read_file_to_string(filename,directory)
    posamezni_oglasi = page_to_ads (stran_s_podatki)
    
    for oglas in posamezni_oglasi : 
        seznam_slovarjev= []
        slovar = get_dict_from_ad_block(oglas)
        seznam_slovarjev.append(slovar)
    
    return seznam_slovarjev

def ads_frontpage():
    return ads_from_file(cat_directory,frontpage_filename)

## Obdelane podatke želimo sedaj shraniti.


Funkcija ***write_csv*** v csv datoteko podano s parametroma *directory/filename* zapiše vrednosti v parametru *rows* pripadajoče ključem podanim v *fieldnames*.

In [13]:
def write_csv(fieldnames, rows, directory, filename):

    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    
    with open(path, 'w') as csv_file:
    
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in rows:
            writer.writerow(row)
    return

## Definirajte funkcijo, ki sprejme neprazen seznam slovarjev, ki predstavljajo podatke iz oglasa mačke, in zapiše vse podatke v csv datoteko. Imena za stolpce *fieldnames* pridobite iz slovarjev.

Funkcija ***write_cats_ads_to_csv*** vse podatke iz parametra *ads* zapiše v csv datoteko podano s parametroma *directory/filename*. 
Funkcija predpostavi, da so ključi vseh sloverjev parametra *ads* enaki in je seznam *ads* neprazen.

Stavek *assert* preveri da zahteva velja.
Če drži, se program normalno izvaja, drugače pa sproži napako.
Prednost je v tem, da ga lahko pod določenimi pogoji izklopimo v produkcijskem okolju.

In [14]:
def write_cat_ads_to_csv(ads, directory, filename):

    assert ads and (all(j.keys() == ads[0].keys() for j in ads))
    raise NotImplementedError()

In [15]:
def write_cat_ads_to_csv(ads, directory, filename):
    '''Write a CSV file containing one ad from "ads" on each row.'''
    write_csv(ads[0].keys(), ads, directory, filename)


def write_cat_csv(ads):
    '''Save "ads" to "cat_directory"/"csv_filename"'''
    write_cat_ads_to_csv(ads, cat_directory, csv_filename)


In [16]:
# Celoten program poženemo v glavni funkciji

def main(redownload=True, reparse=True):
    """Funkcija izvede celoten del pridobivanja podatkov:
    1. Oglase prenese iz bolhe
    2. Lokalno html datoteko pretvori v lepšo predstavitev podatkov
    3. Podatke shrani v csv datoteko
    """
    # Najprej v lokalno datoteko shranimo glavno stran

    # Iz lokalne (html) datoteke preberemo podatke

    # Podatke prebermo v lepšo obliko (seznam slovarjev)

    # Podatke shranimo v csv datoteko

    # Dodatno: S pomočjo parameteov funkcije main omogoči nadzor, ali se
    # celotna spletna stran ob vsakem zagon prense (četudi že obstaja)
    # in enako za pretvorbo

    raise NotImplementedError()


#if __name__ == '__main__':
#    main()